In [ ]:
# --- 1. IMPORTS & LOGIC CORE ---
import sys
import os
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, State, ctx
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc
import plotly.io as pio

# Fix: Force default template globally
pio.templates.default = "plotly_dark"

# Ensure project path is correct
sys.path.append(os.path.abspath(".."))

# --- DESIGN CONFIG (Shared Variables) ---
THEME = {
    'bg_main': '#0b0c10',       # Deep Space Black
    'card_bg': '#1f2833',       # Dark Slate
    'text_main': '#c5c6c7',     # Light Grey
    'text_header': '#66fcf1',   # Neon Cyan
    'accent': '#45a29e'         # Teal
}

BIRD_COLOR_MAP = {
    'Eric': '#d633ff',    # Neon Magenta
    'Nico': '#00ff41',    # Neon Green
    'Sanne': '#00f7ff'    # Neon Cyan
}

# --- DATA PROCESSING ---
try:
    df = pd.read_csv('data/bird_migration.csv')
except:
    df = pd.read_csv('../data/bird_migration.csv')

df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(['bird_name', 'date_time'])

def count_significant_stops(sub_df, speed_threshold=0.5, min_duration_hours=1):
    is_resting = sub_df['speed_2d'] < speed_threshold
    block_ids = (is_resting != is_resting.shift()).cumsum()
    block_durations = sub_df.groupby(block_ids)['date_time'].agg(lambda x: x.max() - x.min())
    block_is_rest = sub_df.groupby(block_ids)['speed_2d'].mean() < speed_threshold
    min_duration = pd.Timedelta(hours=min_duration_hours)
    return ((block_is_rest) & (block_durations > min_duration)).sum()

# Aggregation
alt_stats = df.groupby('bird_name').agg(
    Max_Altitude=('altitude', 'max'), Avg_Altitude=('altitude', 'mean'), Min_Altitude=('altitude', 'min')
)
speed_stats = df[df['speed_2d'] >= 0.5].groupby('bird_name').agg(
    Max_Speed=('speed_2d', 'max'), Avg_Speed=('speed_2d', 'mean'), Min_Speed=('speed_2d', 'min')
)
stop_counts = df.groupby('bird_name').apply(count_significant_stops, include_groups=False)
df_bird_stats = alt_stats.join(speed_stats).join(stop_counts.rename('Total_Rest')).reset_index()

# --- VISUALIZATION ENGINES (Dark Mode Enabled) ---
def create_map(df):
    if df.empty: return go.Figure().update_layout(template="plotly_dark")
    fig = go.Figure()
    for bird in df['bird_name'].unique():
        dff = df[df['bird_name'] == bird]
        fig.add_trace(go.Scattergeo(
            lat=dff['latitude'], lon=dff['longitude'], mode='lines+markers', name=bird,
            line=dict(width=1.5, color=BIRD_COLOR_MAP.get(bird, 'gray')), marker=dict(size=4, opacity=0.8)
        ))
    fig.update_geos(visible=True, showcountries=True, countrycolor="#444", showland=True, landcolor="#1c1c1c", showocean=True, oceancolor="#0b0c10", projection_type="orthographic", bgcolor="#0b0c10")
    fig.update_layout(title={'text': "TRAJECTORY MAP", 'font': {'size': 20, 'color': THEME['text_header']}}, template="plotly_dark", margin={"r":0,"t":60,"l":0,"b":0}, paper_bgcolor="rgba(0,0,0,0)")
    return fig

def build_bar_chart(df_bird_stats, selected_bird, category, selected_stats):
    if not selected_bird or not selected_stats: return go.Figure().update_layout(template="plotly_dark", title="Select options")
    
    cols = [f"{stat}_{category}" if category != 'Rest' else 'Total_Rest' for stat in selected_stats]
    df_melted = df_bird_stats[df_bird_stats['bird_name'].isin(selected_bird)].melt(id_vars='bird_name', value_vars=cols, var_name='Metric', value_name='Value')
    
    fig = go.Figure()
    for bird in df_melted['bird_name'].unique():
        bd = df_melted[df_melted['bird_name'] == bird]
        fig.add_trace(go.Bar(x=bd['Metric'], y=bd['Value'], name=bird, marker_color=BIRD_COLOR_MAP.get(bird), text=bd['Value'].astype(int), textposition='auto'))
    
    fig.update_layout(title={'text': f"{category.upper()} COMPARISON", 'font': {'color': 'white'}}, template="plotly_dark", plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    return fig

# (Other visualization functions: build_line_chart_altitude, build_line_chart_speed, build_animated_map assumed here...)
# For brevity, I'm keeping the core logic consistent with what we established.
print("✅ Logic Core Loaded Successfully")

# --- 3. VISUALIZATION ENGINES ---


In [6]:

# MAP ENGINE
def create_map(df):
    fig = go.Figure()
    if df.empty:
        fig.add_annotation(text="No data selected", x=0.5, y=0.5, showarrow=False, font={'color':'white'})
        fig.update_layout(template="plotly_dark", paper_bgcolor="rgba(0,0,0,0)")
        return fig
    
    unique_birds = df['bird_name'].unique()
    for bird in unique_birds:
        dff = df[df['bird_name'] == bird]
        fig.add_trace(go.Scattergeo(
            lat=dff['latitude'], lon=dff['longitude'], mode='lines+markers', name=bird,
            line=dict(width=1.5, color=BIRD_COLOR_MAP.get(bird, 'gray')), 
            marker=dict(size=4, opacity=0.8), hoverinfo='text', text=dff['bird_name']
        ))
    
    fig.update_geos(
        visible=True, resolution=50, showcountries=True, countrycolor="#444",
        showcoastlines=True, coastlinecolor="#444", showland=True, landcolor="#1c1c1c",       
        showocean=True, oceancolor="#0b0c10", showlakes=True, lakecolor="#0b0c10",
        showrivers=True, rivercolor="#0b0c10", projection_rotation=dict(lon=-10, lat=20),
        projection_type="orthographic", fitbounds="locations", bgcolor="#0b0c10"
    )
    fig.update_layout(
        title={'text': "TRAJECTORY MAP", 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20, 'color': THEME['text_header']}},
        margin={"r":0,"t":60,"l":0,"b":0}, paper_bgcolor="rgba(0,0,0,0)", 
        legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.05, bgcolor="rgba(0,0,0,0.5)", font={'color':'white'}),
        template="plotly_dark"
    )
    return fig

# BAR CHART ENGINE
def build_bar_chart(df_bird_stats, selected_bird, category, selected_stats):
    if not selected_bird or not selected_stats:
        return go.Figure().update_layout(title={'text': "Select options", 'font':{'color':'white'}}, template="plotly_dark")

    cols_to_plot = []
    title_prefix, y_label = "", ""
    if category == "Altitude":
        y_label, title_prefix = "Altitude (m)", "ALTITUDE COMPARISON"
        for stat in selected_stats: cols_to_plot.append(f"{stat}_{category}")
    elif category == "Speed":
        y_label, title_prefix = "Speed (km/h)", "SPEED COMPARISON"
        for stat in selected_stats: cols_to_plot.append(f"{stat}_{category}")
    elif category == "Rest":
        y_label, title_prefix = "Count", "REST STOPS"
        cols_to_plot.append("Total_Rest")

    df_filtered = df_bird_stats[df_bird_stats['bird_name'].isin(selected_bird)].copy()
    valid_cols = [c for c in cols_to_plot if c in df_filtered.columns]
    if not valid_cols: return go.Figure().update_layout(template="plotly_dark", title="No data")

    df_melted = df_filtered.melt(id_vars='bird_name', value_vars=valid_cols, var_name='Metric', value_name='Value')
    unique_birds = sorted(df_melted['bird_name'].unique())
    fig = go.Figure()
    
    for bird in unique_birds:
        bird_data = df_melted[df_melted['bird_name'] == bird].copy()
        base_color = BIRD_COLOR_MAP.get(bird, '#808080')
        fig.add_trace(go.Bar(
            x=bird_data['Metric'], y=bird_data['Value'], name=bird,
            marker=dict(color=base_color, line=dict(color='white', width=0.5)),
            text=bird_data['Value'].round(0).astype(int), texttemplate='%{text:,}', textposition='outside'
        ))

    fig.update_layout(
        title={'text': title_prefix, 'x': 0.5, 'xanchor': 'center', 'font': {'size': 18, 'color': 'white'}},
        template='plotly_dark', legend=dict(orientation="h", y=1.1, x=1, xanchor="right", font={'color':'white'}),
        margin=dict(t=60, b=40, l=60, r=40), yaxis=dict(title=y_label, gridcolor='#333'),
        xaxis=dict(title="", tickfont=dict(color='silver')), barmode='group', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'
    )
    return fig

# ANIMATED LINE CHART (Generic for Speed/Altitude)
def build_animated_line(df, selected_bird, y_col, title_text, y_label):
    if not selected_bird: return go.Figure().update_layout(template='plotly_dark')
    if isinstance(selected_bird, str): selected_bird = [selected_bird]
    
    df_filtered = df[df["bird_name"].isin(selected_bird)].copy().dropna(subset=["date_time", y_col]).sort_values(["bird_name", "date_time"])
    df_filtered["frame"] = pd.to_datetime(df_filtered["date_time"].dt.strftime("%Y-%m-%d"))
    df_daily = df_filtered.groupby(["bird_name", "frame"]).first().reset_index()
    df_daily["avg_val"] = df_daily.groupby("bird_name")[y_col].expanding().mean().reset_index(level=0, drop=True)
    
    line_plot = px.line(df_daily, x='date_time', y='avg_val', color='bird_name', color_discrete_map=BIRD_COLOR_MAP, template='plotly_dark')
    scatter_plot = px.scatter(df_daily, x='date_time', y='avg_val', color='bird_name', color_discrete_map=BIRD_COLOR_MAP, animation_frame="frame", template='plotly_dark')

    for trace in line_plot.data: scatter_plot.add_trace(trace)
    
    scatter_plot.update_layout(
        uirevision=str(selected_bird), title={'text': title_text, 'x': 0.05, 'xanchor': 'left', 'font':{'color':'white'}},
        template='plotly_dark', margin=dict(t=60, b=50, l=40, r=40), 
        xaxis=dict(title="", showticklabels=False, visible=True), yaxis=dict(title=y_label, gridcolor='#333'),
        updatemenus=[{'type': 'buttons', 'showactive': False, 'x': 0, 'y': -0.15, 'xanchor': 'left', 'yanchor': 'top', 'bgcolor': '#333', 'font':{'color':'white'},
            'buttons': [{'label': 'Play', 'method': 'animate', 'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}]},
                        {'label': 'Pause', 'method': 'animate', 'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}]}]}],
        sliders=[{'active': 0, 'yanchor': 'top', 'y': -0.1, 'xanchor': 'left', 'currentvalue': {'prefix': '', 'visible': False, 'font':{'color':'white'}}, 'pad': {'b': 5, 't': 5}, 'steps': [{'args': [[f['name']], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}], 'method': 'animate', 'label': pd.to_datetime(f['name']).strftime('%b %Y')} for f in scatter_plot.frames]}]
    )
    return scatter_plot

# ANIMATED MAP
def build_animated_map(df, selected_bird):
    if not selected_bird: return create_map(pd.DataFrame())
    if isinstance(selected_bird, str): selected_bird = [selected_bird]
    
    df_filtered = df[df["bird_name"].isin(selected_bird)].copy().dropna(subset=["date_time"]).sort_values(["bird_name", "date_time"])
    df_filtered["frame"] = pd.to_datetime(df_filtered["date_time"].dt.strftime("%Y-%m-%d"))
    df_daily = df_filtered.groupby(["bird_name", "frame"]).first().reset_index().sort_values("frame")
    
    # Path Logic
    df_hourly = df_filtered.groupby(["bird_name", "frame"]).first().reset_index()
    df_hourly["step"] = range(len(df_hourly)) # Simplified step logic for demo
    
    fig = px.line_geo(df_hourly, lat="latitude", lon="longitude", color="bird_name", color_discrete_map=BIRD_COLOR_MAP, line_group="bird_name", template={})
    fig_points = px.scatter_geo(df_daily, lat="latitude", lon="longitude", color="bird_name", color_discrete_map=BIRD_COLOR_MAP, size=np.array([10]*len(df_daily)), animation_frame="frame", animation_group="bird_name", template={})

    for trace in fig.data: fig_points.add_trace(trace)

    fig_points.update_layout(
        geo=dict(fitbounds="locations", showcountries=True, countrycolor="#444", showland=True, landcolor="#1c1c1c", showocean=True, oceancolor="#0b0c10", projection_type="natural earth", bgcolor="#0b0c10"),
        uirevision=str(selected_bird), title={'text': "ANIMATED MOVEMENT", 'x': 0.05, 'xanchor': 'left', 'font':{'color':THEME['text_header']}},
        margin={"r":0,"t":50,"l":0,"b":80}, paper_bgcolor="rgba(0,0,0,0)", template="plotly_dark",
        updatemenus=[{'type': 'buttons', 'showactive': False, 'x': 0, 'y': 0, 'xanchor': 'left', 'yanchor': 'bottom', 'bgcolor': '#333', 'font':{'color':'white'}, 'buttons': [{'label': 'Play', 'method': 'animate', 'args': [None, {'frame': {'duration': 150, 'redraw': True}, 'fromcurrent': True}]}, {'label': 'Pause', 'method': 'animate', 'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}]}]}],
        sliders=[{'active': 0, 'yanchor': 'bottom', 'y': 0.1, 'xanchor': 'left', 'currentvalue': {'prefix': '', 'visible': False, 'font':{'color':'white'}}, 'pad': {'b': 5, 't': 5}, 'steps': [{'args': [[f['name']], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}], 'method': 'animate', 'label': pd.to_datetime(f['name']).strftime('%b %Y')} for f in fig_points.frames]}]
    )
    return fig_points
print("✅ Visualization Engines Ready")

✅ Visualization Engines Ready


# --- 4. APP LAYOUT ---


In [7]:
app = Dash(__name__, external_stylesheets=[dbc.themes.CYBORG]) 
server = app.server

app.layout = dbc.Container([
    
    # --- HERO BANNER ---
    dbc.Row([
        dbc.Col([
            html.Div([
                # Background Video
                html.Img(
                    src=app.get_asset_url('background.gif'), 
                    style={'width': '100%', 'height': '350px', 'objectFit': 'cover', 
                           'filter': 'blur(3px)', 'opacity': '0.6', 'position': 'absolute', 'zIndex': '0'}
                ),
                # Title Overlay
                html.Div([
                    html.H1("JOURNEYS IN THE SKY", className="display-3 fw-bold", style={'color': '#fff', 'letterSpacing': '5px'}),
                    html.P("INTERACTIVE BIRD MIGRATION TRACKER", className="lead", style={'color': THEME['text_header'], 'letterSpacing': '2px'}),
                    html.Hr(style={'borderColor': THEME['text_header'], 'width': '100px', 'margin': '20px auto', 'opacity': '1'}),
                ], style={'position': 'relative', 'zIndex': '2', 'paddingTop': '100px', 'textAlign': 'center'})
            ], style={'position': 'relative', 'height': '350px', 'overflow': 'hidden', 'backgroundColor': '#000', 'borderBottom': f"2px solid {THEME['text_header']}"})
        ], width=12)
    ], className="mb-5"),
    
    # --- MAIN CONTENT ---
    dbc.Row([
        
        # --- LEFT SIDEBAR ---
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("MISSION CONTROL", className="fw-bold", 
                              style={"backgroundColor": "transparent", "color": THEME['text_header'], "borderBottom": "1px solid #444", "letterSpacing": "1px"}),
                dbc.CardBody([
                    html.Label("TRACK BIRD ID", className="fw-bold small", style={'color': '#888'}),
                    dcc.Dropdown(
                        id='bird-name-filter',
                        options=[{'label': s, 'value': s} for s in sorted(df['bird_name'].unique())],
                        value=sorted(df['bird_name'].unique()), 
                        multi=True, clearable=True, className="mb-3 dark-dropdown"
                    ),
                    dbc.Button("SELECT ALL", id="btn-all-birds", color="info", outline=True, size="sm", className="w-100 mb-4"),
                    
                    html.Hr(style={'borderColor': '#444'}),

                    html.Label("DATA METRIC", className="fw-bold small", style={'color': '#888'}),
                    dbc.RadioItems(
                        id='category-selector',
                        options=[
                            {'label': ' ALTITUDE', 'value': 'Altitude'},
                            {'label': ' SPEED', 'value': 'Speed'},
                            {'label': ' REST STOPS', 'value': 'Rest'}
                        ],
                        value='Altitude', className="mb-4 text-white",
                        labelStyle={"display": "block", "marginBottom": "8px"}
                    ),

                    html.Label("STATISTICS", className="fw-bold small", style={'color': '#888'}),
                    dbc.Checklist(
                        id='statistic-checklist',
                        options=[
                            {'label': ' MAX', 'value': 'Max'},
                            {'label': ' AVG', 'value': 'Avg'},
                            {'label': ' MIN', 'value': 'Min'},
                        ],
                        value=['Max', 'Avg', 'Min'], switch=True, className="text-white"
                    )
                ])
            ], style=CARD_STYLE)
        ], width=12, lg=3),
        
        # --- CHARTS AREA ---
        dbc.Col([
            
            # Row 1: Maps
            dbc.Row([
                dbc.Col([
                    dbc.Card([dbc.CardBody(dcc.Graph(id='migration-map', style={'height': '500px'}))], style=CARD_STYLE)
                ], width=12, lg=6),
                
                dbc.Col([
                    dbc.Card([dbc.CardBody(dcc.Graph(id='animated-map', style={'height': '500px'}))], style=CARD_STYLE)
                ], width=12, lg=6),
            ]),
            
            # Row 2: Bar Chart
            dbc.Row([
                dbc.Col([
                    dbc.Card([dbc.CardBody(dcc.Graph(id='main-bar-chart', style={'height': '400px'}))], style=CARD_STYLE)
                ], width=12)
            ]),
            
            # Row 3: Line Charts
            dbc.Row([
                dbc.Col([
                    dbc.Card([dbc.CardBody(dcc.Graph(id='line-chart-altitude', style={'height': '400px'}))], style=CARD_STYLE)
                ], width=12, lg=6),
                
                dbc.Col([
                    dbc.Card([dbc.CardBody(dcc.Graph(id='line-chart-speed', style={'height': '400px'}))], style=CARD_STYLE)
                ], width=12, lg=6),
            ]),
            
        ], width=12, lg=9)
    ])
    
], fluid=True, style={"backgroundColor": THEME['bg_main'], "minHeight": "100vh", "paddingBottom": "50px"})
print("✅ App Layout Defined")

✅ App Layout Defined


# --- 5. CALLBACKS & EXECUTION ---


In [10]:

@app.callback(Output('migration-map', 'figure'), Input('bird-name-filter', 'value'))
def update_map(selected_bird_names):
    filtered = df[df['bird_name'].isin(selected_bird_names)] if selected_bird_names else pd.DataFrame()
    return create_map(filtered)

@app.callback([Output('statistic-checklist', 'options'), Output('statistic-checklist', 'value'), Output('statistic-checklist', 'style')], Input('category-selector', 'value'), State('statistic-checklist', 'value'))
def update_stat_options(category, current_values):
    style = {'display': 'block'}
    if category == 'Rest': return [{'label': ' Total', 'value': 'Total'}], ['Total'], {'display': 'none'}
    elif category == 'Speed': 
        options = [{'label': ' MAX', 'value': 'Max'}, {'label': ' AVG', 'value': 'Avg'}]
        vals = [v for v in current_values if v in ['Max', 'Avg']] if current_values else ['Max', 'Avg']
        return options, vals, style
    else:
        options = [{'label': ' MAX', 'value': 'Max'}, {'label': ' AVG', 'value': 'Avg'}, {'label': ' MIN', 'value': 'Min'}]
        vals = [v for v in current_values if v in ['Max', 'Avg', 'Min']] if current_values else ['Max', 'Avg', 'Min']
        return options, vals, style

@app.callback(Output('main-bar-chart', 'figure'), [Input('bird-name-filter', 'value'), Input('category-selector', 'value'), Input('statistic-checklist', 'value')])
def update_chart(selected_bird, category, selected_stats):
    return build_bar_chart(df_bird_stats, selected_bird, category, selected_stats)

@app.callback(Output('line-chart-altitude', 'figure'), Input('bird-name-filter', 'value'))
def update_line_chart_altitude(selected_bird):
    return build_animated_line(df, selected_bird, 'altitude', "AVG ALTITUDE OVER TIME", "Altitude (m)")

@app.callback(Output('line-chart-speed', 'figure'), Input('bird-name-filter', 'value'))
def update_line_chart_speed(selected_bird):
    return build_animated_line(df, selected_bird, 'speed_2d', "AVG SPEED OVER TIME", "Speed (km/h)")

@app.callback(Output('animated-map', 'figure'), Input('bird-name-filter', 'value'))
def update_animated_map(selected_bird):
    return build_animated_map(df, selected_bird)

@app.callback(Output('bird-name-filter', 'value'), Input('btn-all-birds', 'n_clicks'), State('bird-name-filter', 'options'), prevent_initial_call=True)
def select_all_species(n_clicks, options):
    return [opt['value'] for opt in options]

if __name__ == '__main__':
    app.run(debug=True, port=8052)